# Bibliotecas e módulos

In [ ]:
# Instalações
!pip install --upgrade plotly
!pip install -U kaleido

In [ ]:
# Bibliotecas
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import re
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.offline as py
import plotly.graph_objs as go
import plotly
import kaleido
from wordcloud import WordCloud
import plotly.express as px
import io
from PIL import Image
import plotly.subplots as sp
import math
from collections import Counter

In [ ]:
# Importar módulos
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Etapas

## Coleta

In [ ]:
class LetrasMusica:
    def __init__(self):
        self.base_url = "https://www.letras.mus.br/"
        self.session = requests.Session()

    def salvar_letra(self, nome_artista, nome_musica, letra_musica):
        # Substitui caracteres inválidos no nome da música para o sistema de arquivos
        nome_musica = nome_musica.replace("/", "_").replace("\\", "_").replace(":", "-")

        diretorio = f"./letras/{nome_artista}"
        os.makedirs(diretorio, exist_ok=True)  # Cria o diretório se não existir

        caminho_arquivo = os.path.join(diretorio, f"{nome_musica}.txt")
        with open(caminho_arquivo, 'w', encoding='utf-8') as arquivo:
            arquivo.write(letra_musica)

    def extrair_letra(self, url):
        try:
            response = self.session.get(url)
            response.raise_for_status()  # Lança um erro se a requisição falhar

            soup = BeautifulSoup(response.content, 'html.parser')

            # Encontrar o elemento que contém a letra da música
            letra_element = soup.find('div', class_='lyric-original')

            if letra_element:
                # Extrair o texto da letra da música
                letra_musica = letra_element.get_text(separator='\n')
                return letra_musica.strip()
            else:
                print(f"Letra não encontrada para {url}")
                return ""

        except Exception as e:
            print(f"Erro ao extrair letra de {url}: {e}")
            return ""

    def extrair_letras(self, artistas):
        for artista in artistas:
            url_artista = f"{self.base_url}{artista}/"

            try:
                response = self.session.get(url_artista)
                response.raise_for_status()  # Lança um erro se a requisição falhar

                soup = BeautifulSoup(response.content, 'html.parser')

                # Encontrar todas as entradas de músicas
                entradas_musica = soup.find_all('li', class_='songList-table-row --song')

                for entrada in entradas_musica:
                    # Extrair o nome da música e o nome do artista
                    nome_musica = entrada.find('span').text.strip()
                    nome_artista = entrada['data-artist']

                    # Extrair a URL da letra da música
                    url_letra = f"{self.base_url}{entrada.a['href']}"

                    # Extrair a letra da música
                    letra_musica = self.extrair_letra(url_letra)

                    # Salvar a letra da música em um arquivo
                    self.salvar_letra(nome_artista, nome_musica, letra_musica)
                    print(f"Letra salva para: {nome_musica} - {nome_artista}")

            except Exception as e:
                print(f"Erro ao extrair letras do artista {url_artista}: {e}")

## Funções

In [ ]:
def carregar_letras_em_df(diretorio_base='./letras'):
    dados_letras = []

    # Percorrer o diretório base (onde estão as pastas com os artistas)
    for nome_artista in os.listdir(diretorio_base):
        caminho_artista = os.path.join(diretorio_base, nome_artista)

        if os.path.isdir(caminho_artista):  # Verifica se é uma pasta (um artista)
            # Percorrer as músicas dentro da pasta do artista
            for nome_musica in os.listdir(caminho_artista):
                caminho_musica = os.path.join(caminho_artista, nome_musica)

                if nome_musica.endswith('.txt'):  # Verifica se é um arquivo .txt (uma letra)
                    # Abrir o arquivo e ler o conteúdo
                    with open(caminho_musica, 'r', encoding='utf-8') as arquivo:
                        letra_musica = arquivo.read().strip()

                    # Adicionar os dados ao DataFrame
                    dados_letras.append({
                        'artista': nome_artista,
                        'musica': nome_musica.replace('.txt', ''),
                        'letra': letra_musica
                    })

    # Criar o DataFrame com os dados das letras
    df_letras = pd.DataFrame(dados_letras, columns=['artista', 'musica', 'letra'])

    return df_letras

In [ ]:
# Função para "limpeza"
def preprocess_text(text):
    # Remover caracteres especiais e números, deixando apenas palavras
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Converter o texto para letras minúsculas
    text = text.lower()

    # Tokenizar o texto com spaCy
    doc = nlp(text)

    # Lematizar as palavras e remover stop words
    lemmatized_text = ' '.join([token.lemma_ for token in
                                doc if token.text not in
                                stopwords.words('english') and
                                len(token) > 4])

    return lemmatized_text

In [ ]:
# Função para gerar uma imagem de nuvem de palavras a partir do texto
def gerar_imagem_nuvem(texto):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    imagem_buffer = io.BytesIO()
    wordcloud.to_image().save(imagem_buffer, format='PNG')
    imagem_buffer.seek(0)
    return Image.open(imagem_buffer)

In [ ]:
# Função para gerar e exibir as nuvens de palavras organizadas em subplots dois por dois
def gerar_nuvens_em_subplots(df):
    # Contar o número de artistas únicos
    artistas_unicos = df['artista'].nunique()

    # Definir o número de colunas e calcular o número de linhas necessário
    colunas = 2
    linhas = math.ceil(artistas_unicos / colunas)

    # Criar os subplots - várias linhas e duas colunas
    fig = sp.make_subplots(rows=linhas, cols=colunas, subplot_titles=df['artista'].unique())

    # Percorrer cada artista e adicionar uma nuvem de palavras ao subplot correspondente
    for idx, (artista, grupo) in enumerate(df.groupby('artista'), 1):
        # Unir as letras limpas desse artista
        texto_unido = ' '.join(grupo['letrasLimpas'])

        # Gerar a imagem da nuvem de palavras
        imagem = gerar_imagem_nuvem(texto_unido)

        # Converter a imagem para uma figura Plotly
        fig_image = px.imshow(imagem).data[0]

        # Calcular a linha e coluna correspondentes para o subplot
        linha = math.ceil(idx / colunas)
        coluna = idx % colunas if idx % colunas != 0 else colunas

        # Adicionar a imagem ao subplot
        fig.add_trace(fig_image, row=linha, col=coluna)

    # Atualizar o layout para remover os eixos e melhorar o espaçamento
    fig.update_layout(
        height=linhas * 500,  # Ajustar altura com base no número de linhas
        width=colunas * 500,  # Ajustar largura com base no número de colunas
        showlegend=False,
        margin=dict(l=10, r=10, t=40, b=10)
    )

    # Remover as etiquetas dos eixos em todos os subplots
    for i in range(1, artistas_unicos + 1):
        linha = math.ceil(i / colunas)
        coluna = i % colunas if i % colunas != 0 else colunas
        fig.update_xaxes(showticklabels=False, row=linha, col=coluna)
        fig.update_yaxes(showticklabels=False, row=linha, col=coluna)

    # Exibir o gráfico com as nuvens organizadas
    fig.show()

In [ ]:
# Função para contar os termos mais utilizados por cada artista
def contar_termos_mais_frequentes(df, top_n=10):
    # Agrupar as letras limpas por artista
    termos_frequentes_por_artista = {}

    for artista, grupo in df.groupby('artista'):
        # Unir todas as letras limpas desse artista em uma única string
        texto_unido = ' '.join(grupo['letrasLimpas'])

        # Quebrar o texto em palavras e contar a frequência de cada uma
        palavras = texto_unido.split()
        contagem_palavras = Counter(palavras)

        # Selecionar as top N palavras mais frequentes
        termos_frequentes_por_artista[artista] = contagem_palavras.most_common(top_n)

    return termos_frequentes_por_artista

In [ ]:
# Função para gerar gráficos de barras por artista
def gerar_graficos_barras(termos_mais_utilizados):
    # Contar o número de artistas únicos
    artistas_unicos = len(termos_mais_utilizados)

    # Definir o número de colunas e calcular o número de linhas necessário para os subplots
    colunas = 2
    linhas = math.ceil(artistas_unicos / colunas)

    # Criar os subplots
    fig = sp.make_subplots(rows=linhas, cols=colunas, subplot_titles=list(termos_mais_utilizados.keys()))

    # Adicionar os gráficos de barras a cada subplot
    for idx, (artista, termos) in enumerate(termos_mais_utilizados.items(), start=1):
        # Classificar os termos por frequência (maior para menor)
        termos_sorted = sorted(termos, key=lambda x: x[1], reverse=True)

        # Preparar os dados
        termos_labels = [termo[0] for termo in termos_sorted]
        frequencias = [termo[1] for termo in termos_sorted]

        # Criar o gráfico de barras
        trace = go.Bar(
            x=frequencias,
            y=termos_labels,
            orientation='h',  # Barras horizontais
            marker=dict(color='lightskyblue')
        )

        # Calcular a linha e a coluna correspondentes para o subplot
        linha = math.ceil(idx / colunas)
        coluna = (idx - 1) % colunas + 1

        # Adicionar o gráfico de barras ao subplot
        fig.add_trace(trace, row=linha, col=coluna)

        # Inverter a ordem do eixo y para mostrar o maior termo no topo
        fig.update_yaxes(categoryorder='total ascending', row=linha, col=coluna)

    # Atualizar o layout dos subplots
    fig.update_layout(
        height=linhas * 500,  # Ajustar altura com base no número de linhas
        width=colunas * 600,  # Ajustar largura com base no número de colunas
        title_text="Termos Mais Frequentes por Artista (Ordenados do Maior para o Menor)",
        showlegend=False,
        margin=dict(l=40, r=40, t=80, b=40)
    )

    # Exibir o gráfico
    fig.show()

In [ ]:
# Função para calcular a média de palavras por artista
def calcular_media_palavras(df):
    media_palavras_por_artista = {}

    for artista, grupo in df.groupby('artista'):
        total_palavras = sum(grupo['letrasLimpas'].apply(lambda x: len(x.split())))
        media_palavras_por_artista[artista] = total_palavras / len(grupo)

    return media_palavras_por_artista

In [ ]:
# Função para gerar gráfico de média de palavras por artista
def gerar_grafico_media_palavras(media_palavras):
    artistas = list(media_palavras.keys())
    medias = list(media_palavras.values())

    # Criar gráfico de barras
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=artistas,
        y=medias,
        marker=dict(color='lightskyblue')
    ))

    # Atualizar layout
    fig.update_layout(
        title='Média de Palavras por Artista',
        xaxis_title='Artista',
        yaxis_title='Média de Palavras',
        showlegend=False,
        margin=dict(l=40, r=40, t=80, b=40)
    )

    # Exibir o gráfico
    fig.show()

# Mãos na Massa

## Coletar as letras

In [ ]:
# Coletar as letras
if __name__ == "__main__":
    artistas = input("Digite os nomes dos artistas (separados por vírgula): ").split(',')

    letras_musica = LetrasMusica()
    letras_musica.extrair_letras(artistas)

## Pré-processamento

In [ ]:
# Carregar as letras em um DataFrame
df_letras = carregar_letras_em_df()

# Salvar
df_letras.to_excel('letras.xlsx', index=False)

In [ ]:
# Carregar a base
df = pd.read_excel('/content/letras.xlsx')

In [ ]:
# Carregar o modelo de língua em inglês do spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Limpar o texto
df['letrasLimpas'] = df['letra'].apply(preprocess_text)
df

## Analisar

In [ ]:
# Análise de sentimentos
sid = SentimentIntensityAnalyzer()
sentimentos = df.apply(lambda r: sid.polarity_scores(r['letra']), axis=1)

# Adicionar os scores à base original.
d = pd.DataFrame(list(sentimentos))
df = df.join(d)
df.dropna(inplace=True)

In [ ]:
# Filtrar o DataFrame para a década desejada e calcular a média de 'compound' por artista
dados_ordenados = (df.groupby('artista')['compound'].mean()
                   .sort_values(ascending=True))

# Determinar as cores com base nos valores
cores = np.where(dados_ordenados < 0, 'red', 'blue')

# Criar o gráfico de barras
trace = go.Bar(x=dados_ordenados.index,
               y=dados_ordenados.values,
               name='Sentimento',
               marker={'color': cores})
data = [trace]

# Configurar o layout do gráfico com o template escuro e sem grades
layout = go.Layout(
    title=f'Variação por artistas (sentimentos)',
    xaxis={'title': 'Artistas', 'tickangle': -90, 'showgrid': False},
    yaxis={'title': 'Sentimento', 'showgrid': False}
)

# Criar a figura e mostrar o gráfico
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# Chamar a função para gerar e exibir as nuvens de palavras organizadas em dois por dois
gerar_nuvens_em_subplots(df)

In [ ]:
# Chamar a função para obter os termos mais frequentes por artista
termos_mais_utilizados = contar_termos_mais_frequentes(df, top_n=10)

# Chamar a função para gerar os gráficos de barras
gerar_graficos_barras(termos_mais_utilizados)

In [ ]:
# Chamar a função para calcular a média de palavras por artista
media_palavras_por_artista = calcular_media_palavras(df)

# Chamar a função para gerar o gráfico da média de palavras por artista
gerar_grafico_media_palavras(media_palavras_por_artista)